In [1]:
using BenchmarkTools

In [41]:
@time begin # BOILERPLATE IMPORTS AND FUNCTION DEFINITIONS
    
using Petri
using MacroTools
import MacroTools: postwalk, striplines
using ModelingToolkit
import ModelingToolkit: Constant
import Base: ==, ∈, show, ^
using Catlab.Doctrines
import Catlab.Doctrines: ⊗, compose, otimes
using Catlab.WiringDiagrams
using Catlab.Graphics
using Catlab.Graphics.Graphviz
import Catlab.Graphics.Graphviz: Graph, Edge
using SemanticModels
using SemanticModels.ModelTools.CategoryTheory
using SemanticModels.ModelTools.WiringDiagrams
using SemanticModels.ModelTools.PetriModels
using LabelledArrays
using OrdinaryDiffEq
import OrdinaryDiffEq: solve
using Plots
Petri.N(s) = 1 #sum(s[1:end-3])
using SemanticModels.ModelTools.OpenPetris
import Petri: NullPetri
import SemanticModels.ModelTools.OpenModels: OpenModel
import SemanticModels.ModelTools: model
⊗(f::OpenModel,g::OpenModel) = otimes(f,g)
⊚(f::OpenModel,g::OpenModel) = compose(f,g)
^(x::Doctrines.FreeSymmetricMonoidalCategory.Ob{:generator}, n::Int) = foldl(⊗, x for i in 1:n)
^(f::Doctrines.FreeSymmetricMonoidalCategory.Hom{:id}, n::Int64) = foldl(⊗, f for i in 1:n)
conj(a::OpenModel, b::OpenModel) = compose(b,a,b)
conj(a, b) = compose(b,a,b) 
op(f::Model) = Model(f.S, map(f.Δ) do t reverse(t) end)
op(f::OpenModel) = OpenModel(f.dom, op(f.model), f.codom)
wd(ex) = to_wiring_diagram(ex)
function compile(expr::Doctrines.FreeSymmetricMonoidalCategory.Hom{:compose}, ctx::Dict)
    compose(compile.(expr.args, (ctx,)))
end
function compile(expr::Doctrines.FreeSymmetricMonoidalCategory.Hom{:otimes}, ctx::Dict)
    otimes(compile.(expr.args, (ctx,)))
end
function compile(expr, ctx::Dict)
    return ctx[expr]
end

end

  0.001836 seconds (10.49 k allocations: 554.916 KiB)


compile (generic function with 3 methods)

In [12]:
@benchmark begin # MODEL TRANSFORMATIONS

Base.Filesystem.mkpath("img")
X = OpenPetris.X;
spontaneous = OpenModel([1,2], Model([1,2], [(X[1],X[2])]), [1,2])
σ() = OpenModel([1,2], NullPetri(2), [2,1])
birth   = Model([1], [(X[1], 2X[1])])
death   = Model([1, 2], [(X[1], X[2])])
pred(α,β,γ) = Model([1, 2], [(α*X[1] + β*X[2], γ*X[2])])
b = OpenModel([1], birth, [1])
d = OpenModel([1], death, [1])
p(α, β, γ) = OpenModel([1,2], pred(α, β, γ), [1,2])
pdag(α,β,γ) = OpenModel([1,2], Model([1, 2], [(α*X[2] + β*X[1], γ*X[1])]), [1,2])
Xob = Ob(FreeSymmetricMonoidalCategory, :X)
idₓ = id(Xob)
σh = braid(Xob, Xob)
bh = Hom(:birth, Xob,Xob)
dh = Hom(:death, Xob, Xob)
ph = Hom(:predation, Xob^2, Xob^2)
pdagh = Hom(Symbol("p⋆"), Xob^2, Xob^2)
lookuptable = Dict( idₓ=>eye(1), σh=>σ(), bh=>b, dh=>d, ph=>p(1,1,1.15))
set_primitive!(ctx::Dict, hom, f::OpenModel) = begin
    ctx[hom] = f
    return ctx
end
set_primitive(hom, f::OpenModel) = set_primitive!(lookuptable, hom, f)
cur = Hom(:cure, Xob⊗Xob, Xob⊗Xob)
inf = Hom(Symbol("inf¹³₂₃"), Xob^3, Xob^3)
cure = conj(spontaneous, σ())
trinary  = OpenModel([1,2,3], Model([1,2,3], [(X[1]+X[3], X[2]+X[3])]), [1,2,3])
set_primitive(inf, trinary)
set_primitive(ph, p(1,1,1.15))
set_primitive(cur, cure)
set_primitive(inf, trinary)
    
malariah = compose(inf⊗idₓ, idₓ⊗compose(σh⊗idₓ, idₓ⊗σh, σh⊗idₓ), idₓ⊗inf, idₓ⊗conj(σh⊗idₓ, idₓ⊗σh), cur⊗compose(σh, cur, σh))
birdsh = compose(σh⊗idₓ,idₓ⊗ph, σh⊗idₓ, idₓ⊗ph)
vitalsh = bh⊗dh⊗dh⊗bh⊗dh
birdmalh = compose(malariah⊗ idₓ, idₓ^2⊗birdsh, vitalsh)
    
end

BenchmarkTools.Trial: 
  memory estimate:  82.75 KiB
  allocs estimate:  1323
  --------------
  minimum time:     359.003 μs (0.00% GC)
  median time:      445.699 μs (0.00% GC)
  mean time:        502.483 μs (4.38% GC)
  maximum time:     9.413 ms (93.74% GC)
  --------------
  samples:          9868
  evals/sample:     1

In [11]:
@benchmark begin # COMPILE MODEL TO PETRI

birdmal = compile(birdmalh, lookuptable)
    
end

BenchmarkTools.Trial: 
  memory estimate:  202.09 KiB
  allocs estimate:  3808
  --------------
  minimum time:     474.153 μs (0.00% GC)
  median time:      590.275 μs (0.00% GC)
  mean time:        677.818 μs (7.11% GC)
  maximum time:     7.758 ms (87.83% GC)
  --------------
  samples:          7346
  evals/sample:     1

In [9]:
@benchmark begin # COMPILE PETRI TO DIFFEQ

f = Petri.mk_function(birdmal.model)
    
end

BenchmarkTools.Trial: 
  memory estimate:  1.72 MiB
  allocs estimate:  40332
  --------------
  minimum time:     8.203 ms (0.00% GC)
  median time:      9.581 ms (0.00% GC)
  mean time:        10.067 ms (4.42% GC)
  maximum time:     18.646 ms (46.63% GC)
  --------------
  samples:          497
  evals/sample:     1

In [10]:
@benchmark begin # SOLVE DIFFEQ

u0 = @LArray [20,1,0,100,0.2,0,0,0.0] (:X₁, :X₂, :X₃, :X₄, :X₅, :X₆, :X₇, :X₈)
varnames = ["\$S_p\$", "\$I_p\$", "\$I_m\$", "\$S_m\$", "\$B\$", "\$Dp\$", "\$Dm\$", "\$D_b\$"];

βpm = 0.045
βmp = 0.045
ρp = 0.5
ρm = 0.04
ηI = 0.12
ηS = 0.12
νp = 0.006
δp = 1.0
δm = 0.03
νm = 0.1
δb = 0.85
params = [βpm, βmp,ρp,ρm,ηI, ηS, νp, δp, δm, νm, δb]
prob = ODEProblem(f,u0,(0.0, 160.0),params)
sol = solve(prob,Tsit5(), tol=1e-5)
# @show sol[end]

varnames = [:Sp, :Ip, :Im, :Sm, :B, :Dp, :Dm,:Db]
plotvars = [1,2,3,4,5]
plt = plot(sol, vars =plotvars, labels=varnames[plotvars])
ylims!(plt, (0,100))
xlims!(plt, (0,10))
savefig(plt, "img/birdmal_sol_subcritical.png")
plt

end

BenchmarkTools.Trial: 
  memory estimate:  15.96 MiB
  allocs estimate:  732069
  --------------
  minimum time:     52.583 ms (0.00% GC)
  median time:      59.922 ms (6.28% GC)
  mean time:        59.932 ms (4.82% GC)
  maximum time:     67.951 ms (6.94% GC)
  --------------
  samples:          84
  evals/sample:     1